<a href="https://colab.research.google.com/github/matthea711/learning-genai/blob/main/TravelPlannerAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup & Configuration

## 1.1. Installation

In [92]:
!pip install langchain

In [93]:
!pip install langgraph

In [94]:
!pip install -U langchain_openai

## 1.2. API Keys Setup

In [95]:
from google.colab import userdata     # Uses Google Colab's secret feature to fetch API Keys
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')     # OpenAI API Key
os.environ["OPENWEATHER_API_KEY"] = userdata.get('OpenWeather')     # OpenWeather API Key

## 1.3. Import

In [96]:
from langchain_openai import ChatOpenAI
# from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 2. Tool Definitions

## 2.1. Weather Tool

In [97]:
import requests

def get_weather(destination: str):
    api_key = os.environ["OPENWEATHER_API_KEY"]
    url = f"http://api.openweathermap.org/data/2.5/weather?q={destination}&appid={api_key}&units=metric"
    response = requests.get(url).json()

    if "main" in response:
        temp = response["main"]["temp"]
        desc = response["weather"][0]["description"]
        return f"The weather in {destination} is {desc} with a temperature of {temp}°C."
    else:
        return "Weather data could not be retrieved."

# weather_tool = Tool(
#     name="WeatherTool",
#     func=lambda query: get_weather(query),
#     description="Provides current weather for a given destination"
# )

## 2.2. Hotel Finder Tool

In [98]:
def find_hotels(destination: str, budget_level: str):
    hotels = {
        "budget": f"Budget Inn in {destination} - $50/night",
        "standard": f"Comfort Stay in {destination} - $120/night",
        "luxury": f"Grand Getaway in {destination} - $300/night"
    }
    return hotels.get(budget_level.lower(), "No hotel found for selected budget.")

# hotel_tool = Tool(
#     name="HotelSearchTool",
#     func=lambda query: find_hotels(query["destination"], query["budget"]),
#     description="Suggests hotels based on destination and budget level"
# )

## 2.3. Attraction Tool

In [99]:
def suggest_attractions(destination: str, interests: list):
    interest_map = {
        "history": "visiting the Old Town and National Museum.",
        "nature": "exploring the botanical gardens and the river trail.",
        "art": "checking out the local galleries and art walk.",
        "food": "checking out the local market."
    }
    suggestions = [interest_map.get(interest, "No suggestions.") for interest in interests]
    return f"In {destination}, based on your interests you would probably enjoy " + " ".join(suggestions)

# attraction_tool = Tool(
#     name="AttractionTool",
#     func=lambda query: suggest_attractions(query["destination"], query["interests"]),
#     description="Suggests attractions based on user's interests"
# )

# 3. LangGraph Setup

## 3.1. Define State Schema

In [100]:
from langgraph.graph import StateGraph, END

# Define state schema
from typing import TypedDict, List, Optional

class PlannerState(TypedDict):
    destination: Optional[str]
    budget: Optional[str]
    interests: Optional[List[str]]
    weather_info: Optional[str]
    hotel_info: Optional[str]
    attraction_info: Optional[str]
    final_plan: Optional[str]

## 3.2. Node Setup

In [117]:
from langgraph.graph.message import add_messages

# Conditional if weather is not provided
def input_collector(state: PlannerState) -> PlannerState:
    destination = input("🌍 Destination: ")
    budget = input("💰 Budget (budget / standard / luxury): ").strip().lower()
    interests_input = input("🎯 Interests (history, nature, art, food): ").strip()

    weather_info = input("🌤️ (Optional) Enter known weather info or press Enter to skip: ").strip()
    if not weather_info:
        weather_info = None

    interests = [i.strip().lower() for i in interests_input.split(",")]

    return {
        "destination": destination,
        "budget": budget,
        "interests": interests,
        "weather_info": weather_info,
        "hotel_info": None,
        "attraction_info": None,
    }

# Weather Node
def weather_node(state: PlannerState) -> PlannerState:
    if not state.get("weather_info"):
        state["weather_info"] = get_weather(state["destination"])
    return state

# Hotel Node
def hotel_node(state: PlannerState) -> PlannerState:
    if not state.get("hotel_info"):
        state["hotel_info"] = find_hotels(state["destination"], state["budget"])
    return state

# Attraction Node
def attraction_node(state: PlannerState) -> PlannerState:
    if not state.get("attraction_info"):
        state["attraction_info"] = suggest_attractions(state["destination"], state["interests"])
    return state

In [134]:
def final_planner(state: PlannerState) -> PlannerState:
    system_template = (
        "You are a helpful travel agent specializing in personalized itineraries.\n"
        "Destination: {destination}\n"
        "Budget: {budget}\n"
        "Interests: {interests}"
    )
    system_prompt = SystemMessagePromptTemplate.from_template(system_template)

    human_template = (
        "Show the {weather} and {hotel} in list format using emojis before showing an itinerary\n"
        "Then generate a detailed 3-day travel itinerary for {destination}.\n"
        "Use this weather info: {weather}\n"
        "Suggested hotel: {hotel}\n"
        "Things to do: {activities}\n"
        "Make the itinerary fun and practical based on {budget} and {interests}.\n"
        "Use emojis as bullets that varies so that it matches the activity\n"
    )
    human_prompt = HumanMessagePromptTemplate.from_template(human_template)

    prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

    messages = prompt.format_prompt(
        destination=state["destination"],
        budget=state["budget"],
        interests=", ".join(state["interests"]),
        weather=state["weather_info"],
        hotel=state["hotel_info"],
        activities=state["attraction_info"]
    ).to_messages()

    chat = ChatOpenAI(temperature=0.7)
    result = chat(messages)

    return {
        **state,
        "final_plan": result.content
    }


## 3.2. Building the Graph

In [135]:
graph = StateGraph(PlannerState)

# Nodes
graph.add_node("Input", input_collector)
graph.add_node("Weather", weather_node)
graph.add_node("Hotel", hotel_node)
graph.add_node("Attractions", attraction_node)
graph.add_node("Planner", final_planner)

# Edges
graph.set_entry_point("Input")
graph.add_edge("Input", "Weather")
graph.add_edge("Weather", "Hotel")
graph.add_edge("Hotel", "Attractions")
graph.add_edge("Attractions", "Planner")
graph.add_edge("Planner", END)

# Compile the graph
app = graph.compile()

# 4. Run the Agent

In [139]:
print("💁 TRAVEL AGENT 💁\n")
output = app.invoke({})

print("\n")
print(output['final_plan'])

💁 TRAVEL AGENT 💁

🌍 Destination: Thailand
💰 Budget (budget / standard / luxury): standard
🎯 Interests (history, nature, art, food): food, nature, history, art
🌤️ (Optional) Enter known weather info or press Enter to skip: 


🌥️ Weather in Thailand: Overcast clouds, 30.81°C

💤 Comfort Stay in Thailand: $120/night at Comfort Stay

---

**3-Day Travel Itinerary for Thailand**

**Day 1:**
- 🍲 Start your day with a traditional Thai breakfast at a local market.
- 🌿 Explore the lush botanical gardens and immerse yourself in nature.
- 🚶‍♂️ Take a leisurely stroll along the river trail, enjoying the scenic views.
- 🎨 Visit local galleries and take part in an art walk in the afternoon.
- 🍽️ Indulge in a delicious Thai dinner at a local restaurant.

**Day 2:**
- 🕌 Immerse yourself in the rich history of Thailand by visiting the Old Town.
- 🏛️ Explore the National Museum to learn more about the country's heritage.
- 🍜 Enjoy a traditional Thai lunch at a historic restaurant.
- 🚤 Take a boat tour to

In [ ]:
# OLD CODE

# Input Node
# Simulate user input
# def input_collector(state: PlannerState) -> PlannerState:
#     return {
#         **state,
#         "destination": "Paris",
#         "budget": "standard",
#         "interests": ["art", "history"]
#     }

# User input
# def input_collector(state: PlannerState) -> PlannerState:
#     destination = input("🌍 Where are you traveling to? ")

#     budget = ""
#     while budget.lower() not in ["budget", "standard", "luxury"]:
#         budget = input("💰 What's your budget? (budget / standard / luxury): ")

#     interests_input = input("🎯 What are your interests? (comma-separated, e.g., art, history): ")
#     interests = [interest.strip().lower() for interest in interests_input.split(",")]

#     return {
#         "destination": destination,
#         "budget": budget.lower(),
#         "interests": interests,
#         "weather_info": None,
#         "hotel_info": None,
#         "attraction_info": None,
#     }

# # Final Planner
# def final_planner(state: PlannerState) -> PlannerState:
#     plan = f"""
#     ✈️ Trip Plan to {state['destination']}:
#     🌤️ Weather: {state['weather_info']}
#     🏨 Hotel: {state['hotel_info']}
#     🎯 Attractions: {state['attraction_info']}
#     """
#     return {
#         **state,
#         "final_plan": plan
#     }